Python version : 3.10.7  
Tensorflow version : 2.13.0  
Sklearn version : 1.3.0  
F1 Score: 0.8685714285714285

In [3]:
import pandas as pd
import numpy as np

In [4]:
import tensorflow as tf
import sklearn

print(tf.__version__)
print(sklearn.__version__)

2.13.0
1.3.0


In [5]:
df = pd.read_excel('Raisin_Dataset.xlsx')
df.head()

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,Extent,Perimeter,Class
0,87524,442.246011,253.291155,0.819738,90546,0.758651,1184.040,Kecimen
1,75166,406.690687,243.032436,0.801805,78789,0.684130,1121.786,Kecimen
2,90856,442.267048,266.328318,0.798354,93717,0.637613,1208.575,Kecimen
3,45928,286.540559,208.760042,0.684989,47336,0.699599,844.162,Kecimen
4,79408,352.190770,290.827533,0.564011,81463,0.792772,1073.251,Kecimen


In [6]:
missing_values = df.isna().sum()
print (missing_values)

df = df.dropna()

Area               0
MajorAxisLength    0
MinorAxisLength    0
Eccentricity       0
ConvexArea         0
Extent             0
Perimeter          0
Class              0
dtype: int64


In [7]:
class_counts = df.groupby('Class').size()
print(class_counts)

Class
Besni      450
Kecimen    450
dtype: int64


In [8]:
binaryClasses = {k:v for k, v in zip(list(set(df.iloc[:, -1].tolist())), 
                    list(range(len(list(set(df.iloc[:, -1].tolist()))))))}

In [9]:
binaryClasses

{'Besni': 0, 'Kecimen': 1}

In [10]:
def SubClasses(df):
    classes = binaryClasses[df['Class']]
    return classes

ListForClasses = df.apply(SubClasses, axis=1)
df['Class'] = ListForClasses

In [11]:
df

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,Extent,Perimeter,Class
0,87524,442.246011,253.291155,0.819738,90546,0.758651,1184.040,1
1,75166,406.690687,243.032436,0.801805,78789,0.684130,1121.786,1
2,90856,442.267048,266.328318,0.798354,93717,0.637613,1208.575,1
3,45928,286.540559,208.760042,0.684989,47336,0.699599,844.162,1
4,79408,352.190770,290.827533,0.564011,81463,0.792772,1073.251,1
...,...,...,...,...,...,...,...,...
895,83248,430.077308,247.838695,0.817263,85839,0.668793,1129.072,0
896,87350,440.735698,259.293149,0.808629,90899,0.636476,1214.252,0
897,99657,431.706981,298.837323,0.721684,106264,0.741099,1292.828,0
898,93523,476.344094,254.176054,0.845739,97653,0.658798,1258.548,0


In [12]:
x = np.array(df.iloc[:, 0:7])
y = df.iloc[:, -1]

In [13]:
x

array([[8.75240000e+04, 4.42246011e+02, 2.53291155e+02, ...,
        9.05460000e+04, 7.58650579e-01, 1.18404000e+03],
       [7.51660000e+04, 4.06690687e+02, 2.43032436e+02, ...,
        7.87890000e+04, 6.84129570e-01, 1.12178600e+03],
       [9.08560000e+04, 4.42267048e+02, 2.66328318e+02, ...,
        9.37170000e+04, 6.37612812e-01, 1.20857500e+03],
       ...,
       [9.96570000e+04, 4.31706981e+02, 2.98837323e+02, ...,
        1.06264000e+05, 7.41098519e-01, 1.29282800e+03],
       [9.35230000e+04, 4.76344094e+02, 2.54176054e+02, ...,
        9.76530000e+04, 6.58798253e-01, 1.25854800e+03],
       [8.56090000e+04, 5.12081774e+02, 2.15271976e+02, ...,
        8.91970000e+04, 6.32019963e-01, 1.27286200e+03]])

In [14]:
y

0      1
1      1
2      1
3      1
4      1
      ..
895    0
896    0
897    0
898    0
899    0
Name: Class, Length: 900, dtype: int64

In [15]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=423)

In [16]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.utils import to_categorical

# 데이터 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 클래스를 one-hot 인코딩
y_train_encoded = to_categorical(y_train)
y_test_encoded = to_categorical(y_test)

In [17]:
from autokeras import StructuredDataClassifier
from sklearn.metrics import f1_score
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import Callback

class F1ScoreCallback(Callback):
    def __init__(self, validation_data):
        super().__init__()
        self.validation_data = validation_data
        self.best_f1 = 0  

    def on_epoch_end(self, epoch, logs=None):
    
        y_pred = self.model.predict(self.validation_data[0])


        y_pred_binary = np.round(y_pred)


        f1 = f1_score(self.validation_data[1], y_pred_binary)


        if f1 > self.best_f1:
            self.best_f1 = f1
            self.model.save('best_model.h5')


clf = StructuredDataClassifier(max_trials=50)

f1_callback = F1ScoreCallback(validation_data=(x, y))
clf.fit(X_train, y_train, epochs=100, callbacks=[f1_callback])

best_model = tf.keras.models.load_model('best_model.h5')

_, accuracy = best_model.evaluate(X_test, y_test)
print(f"Test accuracy: {accuracy}")

y_pred = best_model.predict(X_test)
y_pred_binary = np.round(y_pred)
f1 = f1_score(y_test, y_pred_binary)
print(f"Test F1 score: {f1}")



Using TensorFlow backend
INFO:tensorflow:Reloading Tuner from .\structured_data_classifier\tuner0.json
INFO:tensorflow:Oracle triggered exit
Epoch 1/100
23/23 [==============================] - 1s 10ms/step - loss: 0.5380 - accuracy: 0.7819
Epoch 2/100
23/23 [==============================] - 0s 5ms/step - loss: 0.4084 - accuracy: 0.8722
Epoch 3/100
 1/23 [>.............................] - ETA: 0s - loss: 0.2844 - accuracy: 0.9062

c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


23/23 [==============================] - 0s 6ms/step - loss: 0.3642 - accuracy: 0.8625
Epoch 4/100
23/23 [==============================] - 0s 5ms/step - loss: 0.3539 - accuracy: 0.8667
Epoch 5/100
 1/23 [>.............................] - ETA: 0s - loss: 0.2685 - accuracy: 0.8750

c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


23/23 [==============================] - 0s 5ms/step - loss: 0.3559 - accuracy: 0.8681
Epoch 6/100
23/23 [==============================] - 0s 5ms/step - loss: 0.3404 - accuracy: 0.8764
Epoch 7/100
23/23 [==============================] - 0s 5ms/step - loss: 0.3589 - accuracy: 0.8764
Epoch 8/100
23/23 [==============================] - 0s 5ms/step - loss: 0.3441 - accuracy: 0.8694
Epoch 9/100
23/23 [==============================] - 0s 5ms/step - loss: 0.3499 - accuracy: 0.8722
Epoch 10/100
23/23 [==============================] - 0s 5ms/step - loss: 0.3405 - accuracy: 0.8722
Epoch 11/100
23/23 [==============================] - 0s 5ms/step - loss: 0.3432 - accuracy: 0.8764
Epoch 12/100
23/23 [==============================] - 0s 5ms/step - loss: 0.3401 - accuracy: 0.8708
Epoch 13/100
23/23 [==============================] - 0s 5ms/step - loss: 0.3480 - accuracy: 0.8750
Epoch 14/100
23/23 [==============================] - 0s 5ms/step - loss: 0.3417 - accuracy: 0.8792
Epoch 15/100
23/2

...

23/23 [==============================] - 0s 6ms/step - loss: 0.2869 - accuracy: 0.8819
Epoch 98/100
23/23 [==============================] - 0s 5ms/step - loss: 0.2858 - accuracy: 0.8833
Epoch 99/100
 1/23 [>.............................] - ETA: 0s - loss: 0.1700 - accuracy: 0.9688

c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


23/23 [==============================] - 0s 5ms/step - loss: 0.2791 - accuracy: 0.8833
Epoch 100/100
23/23 [==============================] - 0s 5ms/step - loss: 0.2882 - accuracy: 0.8708
INFO:tensorflow:Assets written to: .\structured_data_classifier\best_model\assets


INFO:tensorflow:Assets written to: .\structured_data_classifier\best_model\assets


6/6 [==============================] - 0s 1ms/step - loss: 0.3204 - accuracy: 0.8722
Test accuracy: 0.8722222447395325
6/6 [==============================] - 0s 1ms/step
Test F1 score: 0.8685714285714285


In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, SimpleRNN, LSTM
from sklearn.metrics import f1_score
# 컨볼루션 레이어 모델 
model = Sequential()
model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=(X_train_scaled.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(2, activation='softmax')) 


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


X_train_reshaped = X_train_scaled.reshape(X_train_scaled.shape[0], X_train_scaled.shape[1], 1)
X_test_reshaped = X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1], 1)


model.fit(X_train_reshaped, y_train_encoded, epochs=100, batch_size=16)

y_pred = model.predict(X_test_reshaped)
y_pred_classes = np.argmax(y_pred, axis=1)
f1 = f1_score(np.argmax(y_test_encoded, axis=1), y_pred_classes)
print(f'F1 Score: {f1}')


Epoch 1/100
45/45 [==============================] - 1s 1ms/step - loss: 0.4750 - accuracy: 0.8292
Epoch 2/100
45/45 [==============================] - 0s 1ms/step - loss: 0.3649 - accuracy: 0.8542
Epoch 3/100
45/45 [==============================] - 0s 1ms/step - loss: 0.3488 - accuracy: 0.8736
Epoch 4/100
45/45 [==============================] - 0s 1ms/step - loss: 0.3436 - accuracy: 0.8667
Epoch 5/100
45/45 [==============================] - 0s 1ms/step - loss: 0.3521 - accuracy: 0.8681
Epoch 6/100
45/45 [==============================] - 0s 1ms/step - loss: 0.3400 - accuracy: 0.8736
Epoch 7/100
45/45 [==============================] - 0s 1ms/step - loss: 0.3388 - accuracy: 0.8708
Epoch 8/100
45/45 [==============================] - 0s 1ms/step - loss: 0.3366 - accuracy: 0.8764
Epoch 9/100
45/45 [==============================] - 0s 1ms/step - loss: 0.3372 - accuracy: 0.8778
Epoch 10/100
45/45 [==============================] - 0s 977us/step - loss: 0.3351 - accuracy: 0.8764
Epoch 1

In [19]:
# RNN 모델
model = Sequential()
model.add(SimpleRNN(64, activation='relu', input_shape=(X_train_scaled.shape[1], 1)))
model.add(Dense(32, activation='relu'))
model.add(Dense(2, activation='softmax')) 


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


X_train_reshaped = X_train_scaled.reshape(X_train_scaled.shape[0], X_train_scaled.shape[1], 1)
X_test_reshaped = X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1], 1)


model.fit(X_train_reshaped, y_train_encoded, epochs=100, batch_size=16)

y_pred = model.predict(X_test_reshaped)
y_pred_classes = np.argmax(y_pred, axis=1)
f1 = f1_score(np.argmax(y_test_encoded, axis=1), y_pred_classes)
print(f'F1 Score: {f1}')


Epoch 1/100
45/45 [==============================] - 1s 1ms/step - loss: 0.4837 - accuracy: 0.8194
Epoch 2/100
45/45 [==============================] - 0s 1ms/step - loss: 0.3546 - accuracy: 0.8667
Epoch 3/100
45/45 [==============================] - 0s 1ms/step - loss: 0.3528 - accuracy: 0.8694
Epoch 4/100
45/45 [==============================] - 0s 1ms/step - loss: 0.3366 - accuracy: 0.8764
Epoch 5/100
45/45 [==============================] - 0s 1ms/step - loss: 0.3406 - accuracy: 0.8736
Epoch 6/100
45/45 [==============================] - 0s 1ms/step - loss: 0.3287 - accuracy: 0.8736
Epoch 7/100
45/45 [==============================] - 0s 1ms/step - loss: 0.3281 - accuracy: 0.8639
Epoch 8/100
45/45 [==============================] - 0s 1ms/step - loss: 0.3229 - accuracy: 0.8694
Epoch 9/100
45/45 [==============================] - 0s 1ms/step - loss: 0.3237 - accuracy: 0.8694
Epoch 10/100
45/45 [==============================] - 0s 1ms/step - loss: 0.3182 - accuracy: 0.8736
Epoch 11/

In [20]:
# LSTM 모델
model = Sequential()
model.add(LSTM(64, activation='relu', input_shape=(X_train_scaled.shape[1], 1)))
model.add(Dense(32, activation='relu'))
model.add(Dense(2, activation='softmax')) 


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


X_train_reshaped = X_train_scaled.reshape(X_train_scaled.shape[0], X_train_scaled.shape[1], 1)
X_test_reshaped = X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1], 1)


model.fit(X_train_reshaped, y_train_encoded, epochs=100, batch_size=16)


y_pred = model.predict(X_test_reshaped)
y_pred_classes = np.argmax(y_pred, axis=1)
f1 = f1_score(np.argmax(y_test_encoded, axis=1), y_pred_classes)
print(f'F1 Score: {f1}')


Epoch 1/100
45/45 [==============================] - 1s 2ms/step - loss: 0.6009 - accuracy: 0.7583
Epoch 2/100
45/45 [==============================] - 0s 2ms/step - loss: 0.4291 - accuracy: 0.8292
Epoch 3/100
45/45 [==============================] - 0s 2ms/step - loss: 0.4060 - accuracy: 0.8236
Epoch 4/100
45/45 [==============================] - 0s 2ms/step - loss: 0.3934 - accuracy: 0.8319
Epoch 5/100
45/45 [==============================] - 0s 2ms/step - loss: 0.3916 - accuracy: 0.8292
Epoch 6/100
45/45 [==============================] - 0s 2ms/step - loss: 0.3903 - accuracy: 0.8306
Epoch 7/100
45/45 [==============================] - 0s 2ms/step - loss: 0.3863 - accuracy: 0.8347
Epoch 8/100
45/45 [==============================] - 0s 2ms/step - loss: 0.3827 - accuracy: 0.8403
Epoch 9/100
45/45 [==============================] - 0s 2ms/step - loss: 0.3801 - accuracy: 0.8319
Epoch 10/100
45/45 [==============================] - 0s 2ms/step - loss: 0.3782 - accuracy: 0.8361
Epoch 11/